# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:

full_data_rows_list = []
    

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Load data in Apache Cassandra. 

The **event_datafile_new.csv** contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=black>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(auth_provider=auth_provider)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

Creating a keyspace

In [8]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

Setting the keyspace

In [9]:
session.set_keyspace("sparkify")

#### We should create tables that allow to answer the following queries effectively

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_song_library"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                  PRIMARY KEY (sessionId, itemInSession))"

session.execute(query)              

In the first query *sessionId* and *itemInSession* uniquely identify the row. That's why it's a primary key.

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # TODO: It will be pain to support this
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

Let's do a select

In [12]:
query = "SELECT * FROM artist_song_library WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)

for row in rows:
    print(f'{row.artist} -- {row.song} -- {row.length}')

Faithless -- Music Matters (Mark Knight Dub) -- 495.30731201171875


Let's verify that everything is ok

In [13]:
df = pd.read_csv('./event_datafile_new.csv')
df[(df['sessionId'] == 338) & (df['itemInSession'] == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
964,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### Query 2

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user_library"
query = query + "(userId int, sessionId int, itemInSession int, artist text, \
                  song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

session.execute(query)

In [15]:
query = "INSERT INTO artist_song_user_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

for _, row in df.iterrows():
    session.execute(query, (row['userId'], row['sessionId'],
                            row['itemInSession'], row['artist'],
                            row['song'], row['firstName'], row['lastName']))

Our query will have a SELECT specifying *userId* and *sessionId*. They must be in primary key, but also we should have an *itemInSession* as a clustering column, because the result should be sorted by it.

Let's do the select:

In [17]:
query = "SELECT artist, song, firstName, lastName FROM artist_song_user_library WHERE userId=10 AND sessionId=182"
rows = session.execute(query)

for row in rows:
    print(f'{row.artist} -- {row.song} -- {row.firstname} -- {row.lastname}')

Down To The Bone -- Keep On Keepin' On -- Sylvie -- Cruz
Three Drives -- Greece 2000 -- Sylvie -- Cruz
Sebastien Tellier -- Kilometer -- Sylvie -- Cruz
Lonnie Gordon -- Catch You Baby (Steve Pitron & Max Sanna Radio Edit) -- Sylvie -- Cruz


Quick check that everything is alright:

In [18]:
df[(df['userId'] == 10) & (df['sessionId'] == 182)][['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
151,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
152,Three Drives,Greece 2000,Sylvie,Cruz
153,Sebastien Tellier,Kilometer,Sylvie,Cruz
154,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [20]:
query = "CREATE TABLE IF NOT EXISTS user_song_library"
query = query + "(song text, firstName text, lastName text, sessionId int, itemInSession int,\
                  PRIMARY KEY (song, firstName, lastName, sessionId, itemInSession))"
session.execute(query)

In [21]:
query = "INSERT INTO user_song_library (song, firstName, lastName, sessionId, itemInSession)"
query = query + "VALUES (%s, %s, %s, %s, %s)"

for _, row in df.iterrows():
    session.execute(query, (row['song'], row['firstName'],
                            row['lastName'], row['sessionId'], row['itemInSession']))

In [22]:
query = "SELECT firstName, lastName FROM user_song_library WHERE song='All Hands Against His Own'"
rows = session.execute(query)

for row in rows:
    print(f'{row.firstname} -- {row.lastname}')

Jacqueline -- Lynch
Sara -- Johnson
Tegan -- Levine


Quick check that everything is alright.

In [23]:
df[df['song'] == 'All Hands Against His Own'][['firstName', 'lastName']]

,firstName,lastName
219,Sara,Johnson
2442,Jacqueline,Lynch
2646,Tegan,Levine


### Drop the tables before closing out the sessions

In [24]:
session.execute("DROP TABLE IF EXISTS artist_song_library")
session.execute("DROP TABLE IF EXISTS artist_song_user_library")
session.execute("DROP TABLE IF EXISTS user_song_library")

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()